> pipeline chains together mul|tiple steps so that the output of each step is used as inpuut to the next step
>> Pipelines makes it easy to apply the same preprocessing to train&test
- Why - maintaining order of arrays is very difficult
- When - we have to multiple steps at a time liek for traininng and testing


In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.feature_selection import SelectKBest,chi2
from sklearn.tree import DecisionTreeClassifier

In [2]:
df = pd.read_csv('train.csv')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
df.drop(columns=['PassengerId','Name','Ticket','Cabin'],inplace=True)

In [4]:
df.columns

Index(['Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare',
       'Embarked'],
      dtype='object')

In [5]:
df.head(1)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.25,S


In [7]:
df.dtypes,df.isnull().sum()

(Survived      int64
 Pclass        int64
 Sex          object
 Age         float64
 SibSp         int64
 Parch         int64
 Fare        float64
 Embarked     object
 dtype: object,
 Survived      0
 Pclass        0
 Sex           0
 Age         177
 SibSp         0
 Parch         0
 Fare          0
 Embarked      2
 dtype: int64)


- Imputer for Age col as there are many missing vals
- Imputer for Embarked as it has 2 values missing (most frequent option)
- OnehotEncoder for nominal cols (sex ,emabarked)
- Scaling for uniform range for all cols
- feature selection (not taking all cols)

In [12]:
# Step 1 -> train/test/split
X_train,X_test,y_train,y_test = train_test_split(df.drop(columns=['Survived']),
                                                 df['Survived'],
                                                 test_size=0.2,
                                                random_state=42)
X_train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
331,1,male,45.5,0,0,28.5000,S
733,2,male,23.0,0,0,13.0000,S
382,3,male,32.0,0,0,7.9250,S
704,3,male,26.0,1,0,7.8542,S
813,3,female,6.0,4,2,31.2750,S


In [52]:
# first transformer impute
# [2] col no
impute_trf= ColumnTransformer([('impute_age',SimpleImputer(),[2]),
                               ('impute_embark',SimpleImputer(strategy='most_frequent'),[6])]
                              ,remainder='passthrough')
# second transformer encoders

encoder_trf= ColumnTransformer([('embrark_sex_ohe',OneHotEncoder(sparse=False,handle_unknown='ignore'),[1,6]),
                               ],remainder='passthrough')
# third transformer for scaling
scale_trf= ColumnTransformer([('scale_minmax',MinMaxScaler(),slice(0,10)),
                             ] ,remainder='passthrough')
# range(0,8)> actually we know we didn't apply first=drop for encoders transformer
# so at first we had 7 cols now minus sex(as it gets removes only it's data remains) (female,male ->2cols more 
# same for embraked it has three data catgeories so 7-2(bc embarked and sex wil get minus)
# we had 5 and then you 5 cols from encoder of them both it's ten
# so 0-8 cols
# fourth transformer for fature selection
feature_sel_trf=SelectKBest(score_func=chi2,k=8)
# fifth trf for model training
model_trf=DecisionTreeClassifier()
# if we didn't add model transformer then we would do fit_transform instead of fit only

In [46]:
# second transformer encoders

encoder_trf= ColumnTransformer([('embrark_sex_ohe',OneHotEncoder(sparse=False,handle_unknown='ignore'),[1,6]),
                               ],remainder='passthrough')

In [47]:
# third transformer for scaling
scale_trf= ColumnTransformer([('scale_minmax',MinMaxScaler(),slice(0,10)),
                             ] ,remainder='passthrough')
# range(0,8)> actually we know we didn't apply first=drop for encoders transformer
# so at first we had 7 cols now minus sex(as it gets removes only it's data remains) (female,male ->2cols more 
# same for embraked it has three data catgeories so 7-2(bc embarked and sex wil get minus)
# we had 5 and then you 5 cols from encoder of them both it's ten
# so 0-8 cols

In [48]:
# fourth transformer for fature selection
feature_sel_trf=SelectKBest(score_func=chi2,k=8)
# fifth trf for model training
model_trf=DecisionTreeClassifier()

## Creating Pipelines

## Pipeline / make_pipeline
- make_pipelin easier as we don't have to pass the temporaray names for transformers

In [38]:
#pipe=make_pipeline(impute_trf,encoder_trf,scale_trf,feature_sel_trf,model_trf)

In [51]:
impute_trf = ColumnTransformer([
    ('impute_age', SimpleImputer(), [2]),
    ('impute_embarked', SimpleImputer(strategy='most_frequent'), [6])
], remainder='passthrough')

# One hot encoding
encode_trf = ColumnTransformer([
    ('ohe_sex_embarked', OneHotEncoder(sparse=False, handle_unknown='ignore'), [1, 6])
], remainder='passthrough')

# Scaling
scale_trf = ColumnTransformer([
    ('scale', MinMaxScaler(), slice(0, 10))
])

# Feature selection
feature_trf = SelectKBest(score_func=chi2, k=8)

# Train the model
model_trf = DecisionTreeClassifier()

# Create Pipeline
pipe = Pipeline([
    ('trf1', impute_trf),
    ('trf2', encode_trf),
    ('trf3', scale_trf),
    ('trf4', feature_trf),
    ('trf5', model_trf)
])

# Alternate syntax
# pipe = make_pipeline(trf1, trf2, trf3, trf4, trf5)

# Train the pipeline
pipe.fit(X_train, y_train)

C:\Users\DELL\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Pipeline(steps=[('trf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('impute_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('trf2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  [1, 6])])),
                ('trf3',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('trf4',
                 SelectKBest(k=8,
                             score_func=<function chi2 at 0x000001D8B8032520>)),
                ('trf5', DecisionTreeClassifier())])

In [41]:
from sklearn import set_config
set_config(display='diagram')

In [63]:
# [1] pehla imputer  operation [1]> first transformer no
print(pipe.named_steps['trf1'].transformers_[0][1].statistics_)
pipe.named_steps['trf1']

[29.49884615]


ColumnTransformer(remainder='passthrough',
                  transformers=[('impute_age', SimpleImputer(), [2]),
                                ('impute_embarked',
                                 SimpleImputer(strategy='most_frequent'),
                                 [6])])

In [54]:
y_pred=pipe.predict(X_test)

In [55]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.6256983240223464

## Cross-validation using pipeline 

In [56]:
# cv=5 5 diff time train/test split aata ha then wo mean of accuracy
from sklearn.model_selection import cross_val_score
cross_val_score(pipe, X_train, y_train, cv=5, scoring='accuracy').mean()

C:\Users\DELL\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
C:\Users\DELL\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
C:\Users\DELL\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
C:\Users\DELL\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `spa

0.6391214419383433

## GridSearch using Pipeline
- hyperparameter tuning

In [68]:
# as we are sing decsion tree max deoth is a very imp paramtere decides the tree length
params={'trf5__max_depth':[1,2,3,4,5,None]}
## here you should give the actual name of the transformer and then double underscore
# jo pipeline ma naam dia ha 

In [69]:
from sklearn.model_selection import GridSearchCV
grid=GridSearchCV(pipe,params,cv=5,scoring='accuracy')
grid.fit(X_train,y_train)

C:\Users\DELL\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
C:\Users\DELL\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
C:\Users\DELL\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
C:\Users\DELL\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `spa

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('trf1',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('impute_age',
                                                                         SimpleImputer(),
                                                                         [2]),
                                                                        ('impute_embarked',
                                                                         SimpleImputer(strategy='most_frequent'),
                                                                         [6])])),
                                       ('trf2',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('ohe_sex_embarked',
                                                                         OneHotEncoder(handle_unknown='ignore',
                                                                                       sparse=False),
                                                                         [1,
                                                                          6])])),
                                       ('trf3',
                                        ColumnTransformer(transformers=[('scale',
                                                                         MinMaxScaler(),
                                                                         slice(0, 10, None))])),
                                       ('trf4',
                                        SelectKBest(k=8,
                                                    score_func=<function chi2 at 0x000001D8B8032520>)),
                                       ('trf5', DecisionTreeClassifier())]),
             param_grid={'trf5__max_depth': [1, 2, 3, 4, 5, None]},
             scoring='accuracy')

In [70]:
grid.best_score_

0.6391214419383433

In [71]:
grid.best_params_

{'trf5__max_depth': 2}

In [72]:
## yaani ke max_depth should be two then
grid.best_estimator_

Pipeline(steps=[('trf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('impute_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('trf2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  [1, 6])])),
                ('trf3',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('trf4',
                 SelectKBest(k=8,
                             score_func=<function chi2 at 0x000001D8B8032520>)),
                ('trf5', DecisionTreeClassifier(max_depth=2))])

In [73]:
import pickle
pickle.dump(pipe,open('models\pipe.pkl','wb'))

## Prediction using pipeline

In [74]:
import pickle


In [76]:
pipe=pickle.load(open('models\pipe.pkl','rb'))

In [79]:
test_input=np.array([2,'female',32.0,0,0,10.5,'S'],dtype=object).reshape(1,7)

In [80]:
pipe.predict(test_input)

C:\Users\DELL\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but SimpleImputer was fitted with feature names
  warnings.warn(
C:\Users\DELL\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but SimpleImputer was fitted with feature names
  warnings.warn(


array([0], dtype=int64)

In [ ]:
# now it doesn't matter if i chnage something on trian and tehn i don't have to do ti all for testing/prediction
# production ma gharbar nhi hoghi